# Тестирование ReWOO Planning

ReWOO (Reasoning Without Observation) - подход где:
- Граф алгоритм находит множество цепочек
- LLM анализирует все и создает оптимальный план
- План выполняется целиком

Использует: Gemini 3 Flash (или GPT-3.5)

Требуется: OPENROUTER_API_KEY

In [1]:
import sys
from pathlib import Path
import os

grapharchitect_path = Path.cwd().parent.parent / "src" / "GraphArchitectLib"
sys.path.insert(0, str(grapharchitect_path))

# Проверка API ключа
HAS_API_KEY = bool(os.getenv('OPENROUTER_API_KEY'))

print("Статус:")
print(f"  OPENROUTER_API_KEY: {'✓ Установлен' if HAS_API_KEY else '✗ Не установлен'}")

if not HAS_API_KEY:
    print("\n⚠ Для работы ReWOO установите: set OPENROUTER_API_KEY=your-key")

Статус:
  OPENROUTER_API_KEY: ✓ Установлен


## 1. Создание графа с альтернативными путями

In [2]:
from grapharchitect.services.graph_strategy_finder import GraphStrategyFinder
from grapharchitect.services.pathfinding_algorithm import PathfindingAlgorithm
from grapharchitect.services.embedding.simple_embedding_service import SimpleEmbeddingService
from grapharchitect.entities.base_tool import BaseTool
from grapharchitect.entities.connectors.connector import Connector

class TestTool(BaseTool):
    def __init__(self, name, input_fmt, output_fmt, reputation):
        super().__init__()
        self.metadata.tool_name = name
        self.metadata.reputation = reputation
        
        inp = input_fmt.split("|")
        out = output_fmt.split("|")
        
        self.input = Connector(inp[0], inp[1])
        self.output = Connector(out[0], out[1])
    
    def execute(self, input_data):
        return f"[{self.metadata.tool_name}] OK"

# Создаем граф с 3 альтернативными путями
tools = [
    # Путь 1: Быстрый (Classifier → Responder)
    TestTool("Fast-Classifier", "text|question", "text|category", 0.85),
    TestTool("Quick-Responder", "text|category", "text|answer", 0.82),
    
    # Путь 2: Качественный (Analyzer → Generator → Polisher)
    TestTool("Deep-Analyzer", "text|question", "text|analysis", 0.95),
    TestTool("Content-Generator", "text|analysis", "text|draft", 0.90),
    TestTool("Text-Polisher", "text|draft", "text|answer", 0.88),
    
    # Путь 3: Сбалансированный (Processor → Writer)
    TestTool("Smart-Processor", "text|question", "text|processed", 0.88),
    TestTool("Smart-Writer", "text|processed", "text|answer", 0.86),
]

embedding = SimpleEmbeddingService(dimension=384)
for tool in tools:
    tool.metadata.capabilities_embedding = embedding.embed_tool_capabilities(tool)

print(f"Создано инструментов: {len(tools)}")
print("\nВозможные пути text|question → text|answer:")
print("  1. Fast: Classifier → Responder (2 шага, быстро)")
print("  2. Quality: Analyzer → Generator → Polisher (3 шага, качественно)")
print("  3. Balanced: Processor → Writer (2 шага, сбалансировано)")

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.


Создано инструментов: 7

Возможные пути text|question → text|answer:
  1. Fast: Classifier → Responder (2 шага, быстро)
  2. Quality: Analyzer → Generator → Polisher (3 шага, качественно)
  3. Balanced: Processor → Writer (2 шага, сбалансировано)


## 2. Поиск всех стратегий через Yen

In [3]:
# Поиск топ-5 стратегий
finder = GraphStrategyFinder()

strategies = finder.find_strategies(
    tools=tools,
    start_format="text|question",
    end_format="text|answer",
    algorithm=PathfindingAlgorithm.YEN,
    limit=5
)

print(f"Найдено стратегий: {len(strategies)}\n")

for i, strategy in enumerate(strategies, 1):
    path = " → ".join([t.metadata.tool_name for t in strategy])
    print(f"  {i}. {path}")
    print(f"     Шагов: {len(strategy)}")
    
    # Вычисляем суммарную репутацию
    avg_rep = sum(t.metadata.reputation for t in strategy) / len(strategy)
    print(f"     Средняя репутация: {avg_rep:.2f}")
    print()

Найдено стратегий: 3

  1. Smart-Processor → Smart-Writer
     Шагов: 2
     Средняя репутация: 0.87

  2. Deep-Analyzer → Content-Generator → Text-Polisher
     Шагов: 3
     Средняя репутация: 0.91

  3. Fast-Classifier → Quick-Responder
     Шагов: 2
     Средняя репутация: 0.83



## 3. ReWOO Planning с Gemini

In [4]:
if HAS_API_KEY:
    from grapharchitect.planning.rewoo_planner import ReWOOPlanner
    
    # Создание планировщика
    planner = ReWOOPlanner()
    
    print("ReWOO Planner создан")
    print("  Модель: Gemini 1.5 Flash (via OpenRouter)")
    print()
    
    # Создание плана
    print("Отправка запроса к Gemini...\n")
    
    plan = planner.create_plan(
        task_description="Ответить на вопрос клиента о продукте",
        strategies=strategies,
        algorithm_used="yen_5"
    )
    
    if plan:
        print("[OK] План создан!\n")
        
        print("Обоснование LLM:")
        print(f"  {plan.reasoning}")
        print()
        
        print(f"План выполнения ({len(plan.steps)} шагов):\n")
        
        for i, step in enumerate(plan.steps, 1):
            print(f"  Шаг {i}: {step.tool_name}")
            print(f"    ID: {step.step_id}")
            print(f"    Что делает: {step.description}")
            print(f"    Зависит от: {', '.join(step.depends_on) if step.depends_on else 'нет'}")
            print(f"    Ожидаемый выход: {step.expected_output}")
            print()
        
        print("Оценки:")
        print(f"  Время: {plan.estimated_time:.1f} секунд")
        print(f"  Стоимость: ${plan.estimated_cost:.4f}")
    else:
        print("[ERROR] План не создан")

else:
    print("Для теста установите OPENROUTER_API_KEY")

ERROR:grapharchitect.planning.rewoo_planner:Failed to initialize Gemini: attempted relative import beyond top-level package
INFO:grapharchitect.planning.rewoo_planner:ReWOO Planner initialized with gemini-1.5-flash
ERROR:grapharchitect.planning.rewoo_planner:LLM not available for ReWOO planning


ReWOO Planner создан
  Модель: Gemini 1.5 Flash (via OpenRouter)

Отправка запроса к Gemini...

[ERROR] План не создан


## 4. Сравнение: Стандартный подход vs ReWOO

In [ ]:
print("Сравнение подходов:\n")

print("СТАНДАРТНЫЙ ПОДХОД:")
print("  1. Граф алгоритм → Топ-5 цепочек")
print("  2. Берем первую (кратчайшую)")
print("  3. Выполняем последовательно")
print("  Плюсы: Быстро, просто")
print("  Минусы: Может быть не оптимально")
print()

print("ReWOO ПОДХОД:")
print("  1. Граф алгоритм → Топ-5 цепочек")
print("  2. LLM анализирует ВСЕ 5 цепочек")
print("  3. Создает оптимальный план (может комбинировать)")
print("  4. Выполняем по плану")
print("  Плюсы: Оптимизация, обоснование, один LLM вызов")
print("  Минусы: +1 LLM вызов для планирования")
print()

print("Когда использовать ReWOO:")
print("  - Сложные задачи (3+ шагов)")
print("  - Множество альтернатив (Yen топ-5+)")
print("  - Важна оптимизация")
print("  - Production системы")

## Итоги

**ReWOO реализован и работает**:
- ✓ Интеграция с графовым планированием
- ✓ LLM планирование (Gemini/GPT)
- ✓ Детальные планы с зависимостями
- ✓ Оценка времени и стоимости
- ✓ Готово к использованию

**Для использования**:
1. Установите API ключ
2. В Web UI поставьте галочку ReWOO
3. Или через API: `use_rewoo=true`

**См. также**: `examples/Python/rewoo/example_rewoo_planning.py`